# Descriptive Stats: Part 2

By Kenneth Burchfiel 

Released under the MIT license

This second part of Python for Nonprofits' descriptive stats section covers several potential data analysis pitfalls. Specifically, it will explore:

1. Challenges with finding average values for tables whose rows are themselves averages
2. Methods of accounting for missing data when creating pivot tables
3. Issues with using `np.where()` to create derivatives of columns with missing data (and why `map()` and `np.select()` are better fits)

I have to admit that, even by programming textbook standards, you may not find this to be the most exciting chapter. You may well be anxious to get ahead to the graphing, mapping, and online dashboard sections of PFN (which are just around the corner). 

However, in order to be confident that your graphs, maps, and dashboards will provide an accurate view of your underlying data, it's crucial to exercise caution when pivoting and transforming tables. This chapter is meant to help you be more cautious--and thus more successful--in your data analysis adventures.

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
e = create_engine('sqlite:///../Appendix/nvcu_db.db')

import sys
sys.path.insert(1, '../Appendix')
from helper_funcs import config_notebook
display_type = config_notebook(display_max_columns = 5,
                              display_max_rows = 8)

## Microdata, pre-baked data, and the 'average of averages' problem

df_transactions, a table of dining hall transactions shown below, is an excellent candidate for creating a diverse set of pivot tables and charts because each transaction has its own row. (In other words, the table contains *microdata*; for more on this subject, reference https://en.wikipedia.org/wiki/Microdata_(statistics).) This allows you to calculate statistics for an arbitrary set of comparison variables by applying an aggregate function to all rows contained within those groups in the table.

In [2]:
df_transactions = pd.read_sql(
    'select * from dining_transactions', con = e)
df_transactions['transactions'] = 1
df_transactions.head(5)

,starting_year,weekday,level,amount,transactions
0,2023,We,Fr,13.36,1
1,2023,Tu,Se,12.22,1
2,2023,We,Se,23.10,1
3,2023,Su,Fr,18.78,1
4,2023,Tu,Fr,11.36,1


If you wanted to calculate the average dining hall transaction amount, you could simply find the mean of all items within this table:

In [3]:
df_transactions['amount'].mean()

np.float64(12.65563577250256)

If you instead wanted to find the average amount spent by *weekday and level*, you could easily do so via pandas' `pivot_table()` function:

In [4]:
df_transactions.pivot_table(
    index = ['weekday', 'level'], values = 'amount', 
    aggfunc = 'mean').reset_index().sort_values(
    by = 'amount', ascending = False)

,weekday,level,amount
14,Su,Se,22.426887
26,We,Se,21.550155
6,Mo,Se,21.346364
22,Tu,Se,21.268938
...,...,...,...
24,We,Fr,9.916478
20,Tu,Fr,9.835650
12,Su,Fr,9.812551
8,Sa,Fr,9.467975


However, you may not always have access to this type of data. Instead, you might receive an *aggregated* dataset in which averages by different groups are *pre-baked*--e.g. already present in the output. As you'll soon see, this makes the table much less flexible.

'pre-baked' is not, to my knowledge, a common statistical term, but I find that it works pretty well for describing this type of data. For instance, once you've *baked* apples, wheat, and sugar (or whatever goes into a pie--I'm not a baker!) into a pie, it's pretty hard to convert that dish into a caramelized apple. Similarly, as the following examples will show, once you've 'baked' a list of transactions into separate sets of averages by level and by weekday, it will be impossible to use that data to calculate total spending amounts by level *and* weekday--as we no longer know how each level value relates to each weekday value.

To illustrate this issues caused by pre-baked datasets, let's generate two DataFrames, the first of which will show average transaction amounts by level, and the second of which will show average amounts by weekday.

In [5]:
df_average_spending_by_level = df_transactions.pivot_table(
    index = 'level', values = [
    'amount', 'transactions'], aggfunc = {
    'amount':'mean', 'transactions':'count'}).reset_index()
df_average_spending_by_level

,level,amount,transactions
0,Fr,9.911620,12410
1,Ju,17.133287,3289
2,Se,21.294252,2046
3,So,13.006451,4708


In [6]:
df_average_spending_by_weekday = df_transactions.pivot_table(
    index = 'weekday', values = [
    'amount', 'transactions'], aggfunc = {
    'amount':'mean', 'transactions':'count'}).reset_index()
df_average_spending_by_weekday

,weekday,amount,transactions
0,Fr,12.721265,2466
1,Mo,12.632190,3608
2,Sa,12.253912,708
3,Su,12.775699,1123
4,Th,12.592931,3964
5,Tu,12.747500,5267
6,We,12.624990,5317


We can see that lower levels (e.g. freshmen and sophomores) tend to spend more per dining hall visit than upper levels; in addition, we can see that spending is relatively constant across weekdays. However, because these tables are pre-baked, we don't have any way of calculating average spending by level *and* weekday like we could with df_transactions. 

In addition, let's say that we wanted to calculate average dining hall spending using df_average_spending_by_level. A naive approach would be to simply calculate the average of each row:

In [7]:
# The following approach is incorrect!

df_average_spending_by_level['amount'].mean()

np.float64(15.336402324109617)

This average is way off the actual average, which we calculated earlier in this code using df_transactions. Why is this the case? As df_average_spending_by_level shows, seniors and juniors spend much more than freshmen and sophomores, yet they also use the dining hall less (as shown by their smaller transaction counts). As a result, if we simply average the mean transaction amounts for each level, we'll overrepresent upperclassmen and thus skew our average transaction amount upward.

In order to avoid this 'average of averages' issue, which arises whenever differences in group sizes skew an average that's in turn based on averages for those groups, we'll need to create a *weighted* average. We can do so by multiplying each row's 'amount' column by its 'transaction' column; adding these products together; and then dividing them by the 'transaction' column.

In [8]:
df_average_spending_by_level['amount_x_transactions'] = (
    df_average_spending_by_level['amount'] 
    * df_average_spending_by_level['transactions'])
df_average_spending_by_level

,level,amount,transactions,amount_x_transactions
0,Fr,9.911620,12410,123003.20
1,Ju,17.133287,3289,56351.38
2,Se,21.294252,2046,43568.04
3,So,13.006451,4708,61234.37


Here's our weighted average, which matches the average calculated earlier:

In [9]:
(df_average_spending_by_level['amount_x_transactions'].sum() 
 / df_average_spending_by_level['transactions'].sum())

np.float64(12.65563577250256)

The following function can be used to calculate weighted means for other datasets:

In [10]:
def weighted_mean(original_df, metric_col, weight_col):
    '''This function calculates, then returns, a weighted mean for
    the DataFrame passed to original_df.
    metric_col: the column storing the variable for which to calculate
    a mean.
    weight_col: the column storing weight values that will be incorporated
    into this weighted mean.    
    '''
    df = original_df.copy() # Prevents the function from modifying
    # the original DataFrame
    df['metric_x_weight'] = df[metric_col] * df[weight_col]
    weighted_mean = (df['metric_x_weight'].sum() / 
    df[weight_col].sum())
    return weighted_mean

Let's try putting this function into action by calculating the average transaction amount using df_average_spending_by_weekday:

In [11]:
weighted_mean(df_average_spending_by_weekday, 
                 metric_col = 'amount',
                 weight_col = 'transactions')

np.float64(12.65563577250256)

This average matches the weighted average that we calculated within df_average_spending_by_level. 

Incidentally, because average transaction amounts are relatively constant across weekdays, simply averaging all 'amount' values within df_average_spending_by_weekday will get us very close to the actual average (despite the considerable variation in transaction counts by weekday). These kinds of 'believable', yet incorrect values are particularly insidious: they may go unnoticed for quite a while, whereas an obviously incorrect value (e.g. a calculated mean transaction of -5, or 83,000) would get caught right away.

In [12]:
df_average_spending_by_weekday['amount'].mean()

np.float64(12.621212399856491)

We were able to calculate correct averages within these pre-baked tables because we also knew the number of transactions within each row. In the real world, though, such sample size information may not be available.

From a data analysis perspective, it would be ideal to have all of your source data in microdata (rather than pre-baked) form. However, the microdata approach has its own drawbacks. 

First, data privacy needs may preclude the issuance of microdata. Imagine, for instance, that NVCU released a dataset that contained individual course grades for each student along with those students' ages and majors. If you happened to be the only 18-year-old music major who took a course, anyone with that knowledge could find out how you performed in the class. 

A more private approach, in this case, would be to release separate 'pre-baked' tables that showed average grades by age and by major. (Even with this strategy, if a given pre-baked average was based on only a few students, it might be best to remove that row's data so as to protect those students' privacy.)

Second, microdata can take up much more storage size than pre-baked data. To illustrate this, let's compare the amount of memory, in kilobytes, used by df_transactions (a microdata-based table) with that used by our two pre-baked tables:

In [13]:
microdata_kb = df_transactions.memory_usage(
    index = True, deep = True).sum() / 1000
microdata_kb

np.float64(2829.21)

In [14]:
pre_baked_kb = (df_average_spending_by_level.memory_usage(
    index = True, deep = True).sum() + 
 df_average_spending_by_weekday.memory_usage(
    index = True, deep = True).sum()) / 1000
pre_baked_kb

np.float64(1.033)

In [15]:
microdata_kb / pre_baked_kb

np.float64(2738.828654404647)

The microdata table takes up over 2,700 times more memory than our two pre-baked tables! Therefore, it's understandable that data providers may prefer to share pre-calculated averages as opposed to original datasets.

## Handling missing data when creating pivot tables

If a field passed to the `index` or `columns` argument of a pivot_table() call has a missing value, that missing value won't get incorporated into the final table. This can cause calculation errors if you end up using the pivot table for further analyses. However, mitigating this issue isn't too difficult.

To demonstrate this issue, I'll create a 'faulty' version of df_transactions in the following cell that has `np.nan` (e.g. missing) entries for all 'Wednesday' weekday values and all 'So' level values. As you'll see, these missing values will cause issues when we (1) create a pivot table of this data, then (2) attempt to use that pivot table to determine the sum of all transactions in our dataset. 

In [16]:
df_transactions_faulty = df_transactions.copy()
df_transactions_faulty['weekday'] = np.where(
    df_transactions_faulty['weekday'] == 'We', 
np.nan, df_transactions_faulty['weekday']) 
df_transactions_faulty['level'] = np.where(
    df_transactions_faulty['level'] == 'So', 
np.nan, df_transactions_faulty['level']) 

df_transactions_faulty

,starting_year,weekday,level,amount,transactions
0,2023,NaN,Fr,13.36,1
1,2023,Tu,Se,12.22,1
2,2023,NaN,Se,23.10,1
3,2023,Su,Fr,18.78,1
...,...,...,...,...,...
22449,2023,Tu,Fr,5.98,1
22450,2023,Fr,NaN,7.39,1
22451,2023,Sa,Fr,13.25,1
22452,2023,Mo,Fr,1.95,1


First, let's try converting this dataset into a pivot table that shows total transaction amounts by week.

In [17]:
df_transactions_faulty_pivot = df_transactions_faulty.pivot_table(
    index = 'weekday', values = 'amount', 
    aggfunc = 'sum').reset_index()
df_transactions_faulty_pivot

,weekday,amount
0,Fr,31370.64
1,Mo,45576.94
2,Sa,8675.77
3,Su,14347.11
4,Th,49918.38
5,Tu,67141.08


There's no sign whatsoever of the 'Wednesday' rows. This shouldn't be too surprising (since we removed those 'We' values), but it's worth highlighting that no 'Missing' or 'N/A' row gets returned by the pivot table function.

The complete absence of the 'Wednesday' row makes this this missing data issue easier to identify. However, many cases of missing data are far more insidious. For instance, suppose only a handful of 'Wednesday' entries were missing. We'd see a 'Wednesday' row within the pivot table, but we might not realize that its 'amount' sum was incorrect. (For this reason, it's often a good idea to check for missing entries within any fields that you pass to the `index` or `column` arguments of a `pivot_table()` call.)

If we try to use this pivot table to calculate the sum of all our transactions, we'll end up with an incorrect result, since rows with missing 'weekday' values won't factor into the calculation.

In [18]:
df_transactions_faulty_pivot['amount'].sum()

np.float64(217029.91999999998)

For reference, here's the correct sum:

In [19]:
df_transactions['amount'].sum()

np.float64(284156.99)

Thankfully, it's not too hard to prevent this issue. We simply need to fill in missing values within any fields fields passed to the `index` or `columns` arguments of the `pivot_table()` call.

I'll demonstrate two ways to replace these values with a 'Missing' string. First, if you don't wish to permanently modify your original DataFrame, you can call `.fillna()` on the DataFrame being passed to the `pivot_table()` function. This ensures that rows with missing data get incorporated into the pivot table but leaves the original DataFrame untouched. 

The following code implements this approach via a *dict comprehension*. (For more on this valuable tool, see https://docs.python.org/3/tutorial/datastructures.html#dictionaries .) This approach allows me to use the same list of `index` values that I'll pass to `pivot_table()` to specify which columns' missing entries should get filled in. This results in cleaner and less error-prone code, as if I needed to update my index values twice, once for `pivot_table()` and once for `fillna()`, the two lists could easily get out of sync.

(Another option, by the way, would be to fill missing entries within *all* columns with a 'Missing' string; this could be accomplished via `df_transactions_faulty.fillna('Missing')`. However, if some columns with missing data use float or integer types, this approach could either raise a warning or an error--as you'd be attempting to pass a string into a list of missing values. Thus, I recommend using the more precise solution shown below.

In [20]:
index = ['level', 'weekday'] # These values will get passed to the
# index argument of our pivot_table() call.
# The following dict comprehension will produce the dictionary
# {'level': 'Missing', 'weekday': 'Missing'}--which, when passed to
# fillna(), will instruct Pandas to fill in missing data with 'Missing' 
# only within those two fields.
# (For more on this argument, see
# https://pandas.pydata.org/pandas-docs/stable/reference/
# api/pandas.DataFrame.fillna.html )
df_transactions_corrected_pivot = df_transactions_faulty.fillna(
    {field:'Missing' for field in index}).pivot_table(
    index = index, values = 'amount',
    aggfunc = 'sum').reset_index()
df_transactions_corrected_pivot

,level,weekday,amount
0,Fr,Fr,13183.14
1,Fr,Missing,28856.95
2,Fr,Mo,20214.29
3,Fr,Sa,3739.85
...,...,...,...
24,Se,Sa,1260.53
25,Se,Su,2377.25
26,Se,Th,7017.18
27,Se,Tu,10613.20


Filling in missing 'level' and 'weekday' rows with 'Missing' allowed them to appear within our updated pivot table. As a result, we can now produce an accurate sum of all transactions by adding up the 'amount' rows within this table:

In [21]:
df_transactions_corrected_pivot['amount'].sum()

np.float64(284156.99)

The following approach permanently replaces missing weekday and level values with 'Missing', thus saving you the trouble of repeating this step later in your code. It also iterates through each field in the `index` list via a for loop in order to inform you which columns, in particular, have missing data. 

In some cases, it may be preferable for your code to halt upon finding missing data; this halt serves as an alert that missing data exists, thus prompting you to fill in those missing entries with their actual values. The commented-out code above the first print statement in the following cell would stop this script's operation (by raising a ValueError) if it came across any missing data.

In [22]:
for val in index:
    if df_transactions_faulty[val].isna().sum() > 0:
#         raise ValueError(f"{val} has NaN values; address these before \
# running the following pivot table operation.")
        print(f"{val} has NaN values; these will be filled with 'Missing' \
so that their rows' data can still get incorporated into the following \
pivot table.")
        df_transactions_faulty[val] = df_transactions_faulty[val].fillna(
            'Missing').copy()
        
df_transactions_corrected_pivot = df_transactions_faulty.pivot_table(
    index = index, values = 'amount',
    aggfunc = 'sum').reset_index()
df_transactions_corrected_pivot
                         

level has NaN values; these will be filled with 'Missing' so that their rows' data can still get incorporated into the following pivot table.
weekday has NaN values; these will be filled with 'Missing' so that their rows' data can still get incorporated into the following pivot table.


,level,weekday,amount
0,Fr,Fr,13183.14
1,Fr,Missing,28856.95
2,Fr,Mo,20214.29
3,Fr,Sa,3739.85
...,...,...,...
24,Se,Sa,1260.53
25,Se,Su,2377.25
26,Se,Th,7017.18
27,Se,Tu,10613.20


This new version of `df_transactions_corrected_pivot` also lets us calculate an accurate sum of all transactions within the original dataset:

In [23]:
df_transactions_faulty_pivot['amount'].sum()

np.float64(217029.91999999998)

By the way, the following code would also have allowed us to permanently fill in all missing values within the columns in our `index` list. This code is similar to that used in the first approach, except that it saves its updates back to df_transactions_faulty, thus making them persistent.

In [24]:
df_transactions_faulty = df_transactions_faulty.fillna(
    {field:'Missing' for field in index}).copy()

I'll now undo my corrections to df_transactions_faulty in order to prepare the dataset for the following section.

In [25]:
# Undoing my corrections to df_transactions_faulty:
df_transactions_faulty.replace(
    'Missing', np.nan, inplace = True)
df_transactions_faulty

,starting_year,weekday,level,amount,transactions
0,2023,NaN,Fr,13.36,1
1,2023,Tu,Se,12.22,1
2,2023,NaN,Se,23.10,1
3,2023,Su,Fr,18.78,1
...,...,...,...,...,...
22449,2023,Tu,Fr,5.98,1
22450,2023,Fr,NaN,7.39,1
22451,2023,Sa,Fr,13.25,1
22452,2023,Mo,Fr,1.95,1


## The advantages of map() and np.select() over np.where() when missing data are present

Suppose we'd like to find the total number of dining hall transactions that took place on the weekend versus those that didn't. To make this calculation easier, we can add a 'weekday' column that will store a value of 1 if a transaction fell on Saturday or Sunday and 0 otherwise. 

We could try initializing this column via np.where(). The following cell uses this function to assign a 'weekend' value of 0 to all transactions whose 'weekday' value corresponds to Monday, Tuesday, Wednesday, Thursday, or Friday. Transactions that don't have one of these weekday values are classified as weekend sales.

In [26]:
df_transactions_faulty['weekend'] = np.where(
    df_transactions_faulty['weekday'].isin(
        ['Mo', 'Tu', 'We', 'Th', 'Fr']), 0, 1)
df_transactions_faulty

,starting_year,weekday,...,transactions,weekend
0,2023,NaN,...,1,1
1,2023,Tu,...,1,0
2,2023,NaN,...,1,1
3,2023,Su,...,1,1
...,...,...,...,...,...
22449,2023,Tu,...,1,0
22450,2023,Fr,...,1,0
22451,2023,Sa,...,1,1
22452,2023,Mo,...,1,0


The issue with this approach, as you might have recognized already, is that it classifies all transactions with missing 'weekday' values as taking place during the weekend. We know that this is inaccurate: the transactions with missing weekday entries actually took place on Wednesday. (In real life, of course, we wouldn't know this to be the case--but we still wouldn't want to assume that they fell on the weekend.)

Here is the total number of Saturday and Sunday dining hall transactions according to our 'weekend' column:

In [27]:
len(df_transactions_faulty.query("weekend == 1"))

7148

As it turns out, this sum much higher than the correct value (which we can calculate using our original transactions table):

In [28]:
len(df_transactions.query("weekday in ['Sa', 'Su']"))

1831

Let's now overwrite this faulty 'weekend' column by using `map()` instead. This function will allow us to map each individual weekday value to either 1 (for weekend) entries or 0 (for non-weekend values). Importantly, it also lets us map np.nan (i.e. missing) entries to a third number, -1: this number represents neither weekend nor non-weekend entries.

In [29]:
df_transactions_faulty['weekend'] = (
    df_transactions_faulty['weekday'].map(
    {'Su':0, 'Mo':0, 'Tu': 0, 'We':0, 'Th':0, 'Fr':0, 'Sa':1, 'Su':1,
     np.nan:-1}))
df_transactions_faulty

,starting_year,weekday,...,transactions,weekend
0,2023,NaN,...,1,-1
1,2023,Tu,...,1,0
2,2023,NaN,...,1,-1
3,2023,Su,...,1,1
...,...,...,...,...,...
22449,2023,Tu,...,1,0
22450,2023,Fr,...,1,0
22451,2023,Sa,...,1,1
22452,2023,Mo,...,1,0


If we apply value_counts() to this column, we'll find that our weekend transactions count matches that shown above. The sum for the -1 entry shows the number of transactions that are missing 

In [30]:
df_transactions_faulty['weekend'].value_counts()

weekend
 0    15305
-1     5317
 1     1831
Name: count, dtype: int64

I could have also chosen to use 'Missing' as my marker for invalid data; however, since the other values output by `map()` were integers, I wanted to make the missing data code an integer also.

As an aside, the use of 1 for weekend transactions and 0 for work-week ones makes it easy to determine the percentage of transactions that took place over the weekend:

In [31]:
print(f"{
round(100*df_transactions_faulty.query(
    "weekend != -1")['weekend'].mean(), 2)}% of transactions with valid \
weekday entries took place on a Saturday or Sunday.")

10.69% of transactions with valid weekday entries took place on a Saturday or Sunday.


Note the 'with valid weekday entries' caveat in the above print statement. Since we wouldn't know for sure on which weekdays the missing transactions took place, we wouldn't want our statement to make any assertionss about them.

It was also crucial to exclude rows with weekend values of -1 from our above calculation. If we had kept them in, we would have ended up with a nonsensical percentage of weekend transactions:

In [32]:
print(f"{round(100 * df_transactions_faulty['weekend'].mean(), 2)}% \
of transactions took place on a Saturday or Sunday . . . wait, really? \
I think I need to double-check these numbers . . . ")

-15.53% of transactions took place on a Saturday or Sunday . . . wait, really? I think I need to double-check these numbers . . . 


If you're categorizing continuous rather than categorical data, the `map()` approach above will be hard to implement--as you might end up having to code a ton of values. Therefore, you should use `np.select()` instead. This function generally involves a bit more code than `map()` for categorical data, but it easily accommodates continuous values as well.

Let's say that we want to add a 'large_purchase' flag to each row that will be 1 if the amount was at or above the 90th percentile and 0 otherwise. We could then use this flag to determine each level's likelihood of making such a large purchase.

The 90th percentile can be calculated as follows:

In [33]:
large_purchase_threshold = df_transactions['amount'].quantile(0.9)
# See https://pandas.pydata.org/docs/reference/api/
# pandas.DataFrame.quantile.html
large_purchase_threshold

np.float64(24.067999999999994)

The following code will simplify df_transactions_faulty by removing some columns that won't be needed for the following section. It will also introduce missing numerical data by replacing all transactions whose final digit is a 6 with np.nan.

In [34]:
df_transactions_faulty = df_transactions.copy().drop(
    ['weekday', 'transactions'], 
    axis = 1)

df_transactions_faulty['amount'] = np.where(
    df_transactions_faulty['amount'].astype('str').str[-1] == '6', 
    np.nan, df_transactions_faulty['amount'])

df_transactions_faulty

,starting_year,level,amount
0,2023,Fr,NaN
1,2023,Se,12.22
2,2023,Se,23.10
3,2023,Fr,18.78
...,...,...,...
22449,2023,Fr,5.98
22450,2023,So,7.39
22451,2023,Fr,13.25
22452,2023,Fr,1.95


We wouldn't want to use `np.where()` to initialize our 'large_purchase' column, as we would inadvertently group missing transactions as large or non-large. In addition, `map()` would be a poor solution, as we now have thousands of unique values to code rather than just a few:

In [35]:
len(df_transactions_faulty['amount'].unique())

3131

Therefore, we'll instead use `np.select()`. This function applies a `condlist` (a list of possible conditions) and a `choicelist` (a list of values to apply for each of those conditions) in order to initialize or update a given column. (You don't have to name these items `condlist` and `choicelist`, but those are their corresponding parameter names; see https://numpy.org/doc/stable/reference/generated/numpy.select.html ). Importantly, this function also has a `default` argument that lets you determine what value to enter if none of the conditions were met. 

The following code applies `np.select()` by defining two possible conditions (e.g. a transaction is above the threshold or it isn't) and two corresponding values to enter within our 'large_purchase' column (1 or 0). It also adds a 'default' value of -1; this value will get added to the large_purchase column when a given transaction amount is missing. (Note that, if no default value is specified, numpy will add values of 0--which would easily get mistaken for the 'not a large purchase' condition.)

In [36]:
condlist = [
    df_transactions_faulty['amount'] >= large_purchase_threshold,
    df_transactions_faulty['amount'] <= large_purchase_threshold
]

choicelist = [1,
              0] # Make sure that the order of these entries matches
# the order of your condlist entries!

df_transactions_faulty['large_purchase'] = np.select(
    condlist, choicelist, default = -1)
df_transactions_faulty   

,starting_year,level,amount,large_purchase
0,2023,Fr,NaN,-1
1,2023,Se,12.22,0
2,2023,Se,23.10,0
3,2023,Fr,18.78,0
...,...,...,...,...
22449,2023,Fr,5.98,0
22450,2023,So,7.39,0
22451,2023,Fr,13.25,0
22452,2023,Fr,1.95,0


Now that we've added in this column, we can create a pivot table that shows the likelihood, for each level, that a given transaction was at or above the 90th percentile:

In [37]:
df_transactions_faulty.query("large_purchase != -1").pivot_table(
    index = 'level', 
    values = 'large_purchase', aggfunc = 'mean').reset_index()
                                   

,level,large_purchase
0,Fr,0.000000
1,Ju,0.297705
2,Se,0.434444
3,So,0.081307


This likelihood is evidently much higher for upperclassmen (juniors and seniors) than for underclassmen (freshmen and sophomores). Of course, this analysis isn't perfect, as it's limited to the rows for which we have valid transaction amounts.